In [33]:
import torch
tr_embeddings = []

for i in range(3):
    tr_embeddings.append(torch.load(f'tr_cosmos_embeddings_{i}.pt'))

tr_embeddings = torch.cat(tr_embeddings)

print(tr_embeddings.shape)

torch.Size([30158, 768])


/var/folders/gs/crm1zcvj00j1kl0l1lvz4qf40000gn/T/ipykernel_26030/2008816879.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  tr_embeddings.append(torch.load(f'tr_cosmos_e

In [34]:
import pandas as pd
from codes.pre_processor import PreProcessor

df = pd.read_csv('new_df.csv')

preprocessor = PreProcessor(df)
df = preprocessor.preprocess()
df


/Users/alibayram/Desktop/quality_driven_learning_rate/codes/pre_processor.py:147: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'acil-tip' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['doctor_speciality'].isin(speciality_mapping[key]), 'doctor_speciality'] = key
/Users/alibayram/Desktop/quality_driven_learning_rate/codes/pre_processor.py:154: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'dr-ogr-uyesi' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['doctor_title'].isin(title_mapping[key]), 'doctor_title'] = key


,doctor_title,doctor_speciality,text
0,7,0,"Merhaba, Yaklaşık 4 aydır sağ omuzumda kolumu ..."
1,6,0,Hocam merhaba 29 yaşında erkek hasta hareketle...
2,6,0,"Meraba hocam sorum şu , bir erkek bayıltılıp a..."
3,6,1,Merhaba doktor hanim ben 3 yildir evliyim. Hic...
4,6,1,Merhaba.. Benim size sorum olacakti .. Ben 3 s...
...,...,...,...
124743,6,0,Merhaba L3 4 L4 5 duzeyınde anular buldıng ızl...
124744,6,1,"hocam merhaba , son kontrolüme göre gebelik ha..."
124745,6,0,Hocam 15 yildir basur sikintisi çeken biriyim ...
124746,6,1,hocam öncelikle meraba ortalama 3 yıl önce kad...


In [35]:
from transformers import AutoTokenizer

tr_tokenizer = AutoTokenizer.from_pretrained("alibayram/tr_tokenizer")
tr_tokenizer.is_fast

True

In [36]:
import numpy as np # linear cebir

def embed_text(text):
    # the average of the embeddings of the tokens
    input_ids = tr_tokenizer(text)['input_ids']
    embeddings = []
    for i in input_ids:
        embeddings.append(tr_embeddings[i])
        
    list_of_embeddings = torch.stack(embeddings)
    # return means as list
    return torch.mean(list_of_embeddings, axis=0).detach().numpy()

embedded_text = embed_text("Merhaba, benim adım Ali. Bugün hava çok güzel.")
embedded_text.shape

(768,)

In [37]:
# ReLU fonksiyonu gelen değer 0'dan küçükse 0'a eşitler, diğer durumlarda değeri olduğu gibi bırakır.
def ReLU(degerler):
    degerler[degerler < 0] = 0
    return degerler

In [38]:

def tahmin_et(ornek, agirliklar, biaslar):
    birinci_katman = np.matmul(ornek, agirliklar[0])
    birinci_katman = birinci_katman + biaslar[0]
    birinci_katman = ReLU(birinci_katman)
    
    ikinci_katman = np.matmul(birinci_katman, agirliklar[1])
    
    # Üçüncü katmanın çıktısını hesaplamak için softmax fonksiyonunu kullanıyoruz.
    # Softmax fonksiyonu, çıktıları olasılık değerlerine dönüştürür. Şu şekilde çalışır:
    # 1. İkinci katmandan gelen değerlerin exp() fonksiyonunu alır.
    # 2. Bu değerleri toplar.
    # 3. Her bir değeri toplam değere böler.
    # Bu işlem sonucunda, her bir çıktı değeri 0 ile 1 arasında bir değer alır ve toplam değer 1 olur.
    softmax_pay = np.exp(ikinci_katman)
    softmax_payda = np.sum(softmax_pay, axis=1).reshape(-1,1)
    softmax = softmax_pay / softmax_payda

    return softmax

In [39]:
def kayip_hesapla(y_tahmin, y_gercek):
    '''
    Kayıp fonksiyonu, modelin tahminlerinin ne kadar yanlış olduğunu ölçer.
    Bu örnekte, çok sınıflı çapraz entropi kaybını kullanıyoruz.
    Şu şekilde çalışır:
    1. Her bir örneğin gerçek etiketinin one-hot encoding'ini alır.
    2. Modelin tahminlerini logaritma alır.
    3. Her bir örneğin kaybını hesaplar.
    4. Tüm örneklerin kaybının ortalamasını alır.    
    '''
    global sinif_sayisi
    sinif_sayisi = 2
    ornek_sayisi = len(y_gercek)

    # One-hot encoding
    y_gercek_one_hot = (y_gercek[:, np.newaxis] == np.arange(sinif_sayisi))
    # Şu şekilde bir matris oluşturur:
    # [[0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    #  [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    #  [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    #  ...
    #  [0, 0, 0, 0, 0, 0, 0, 0, 1, 0]]

    # Her bir örneğin kaybını hesaplar
    kayip_ornek = (np.log(y_tahmin) * y_gercek_one_hot).sum(axis=1)
    # Tüm örneklerin kaybının ortalamasını alır
    return -np.mean(kayip_ornek)

In [69]:
def geri_yayilim(agirliklar, biaslar, girdi_verisi, etiketler, duzenleme_katsayisi=1e-4, guvenilirlik=None):
    '''
    Bu fonksiyon, modelin ileri geçişini ve geri yayılım işlemlerini gerçekleştirerek 
    ağırlık ve bias türevlerini hesaplar. Güvenilirlik katsayılarına göre ölçeklenmiş
    türevleri de döndürür.
    '''
    numune_sayisi = girdi_verisi.shape[0]
    
    ### Adım 1: İleri Geçiş
    # Giriş Katmanı -> Gizli Katman
    gizli_katman_giris = np.matmul(girdi_verisi, agirliklar[0]) + biaslar[0]
    gizli_katman_aktivasyonu = np.maximum(0, gizli_katman_giris)  # ReLU aktivasyon fonksiyonu

    # Gizli Katman -> Çıkış Katmanı
    cikis_katman_giris = np.matmul(gizli_katman_aktivasyonu, agirliklar[1])
    ekspotansiyel = np.exp(cikis_katman_giris)
    toplam_ekspotansiyel = np.sum(ekspotansiyel, axis=1).reshape(-1, 1)
    cikis_aktivasyonu = ekspotansiyel / toplam_ekspotansiyel
    
    ### Adım 2: Geri Yayılım
    # Çıkış katmanından gizli katmana türev hesaplama
    etiketler_bir_sicak_vektor = (etiketler[:, np.newaxis] == np.arange(cikis_aktivasyonu.shape[1]))
    delta2 = (cikis_aktivasyonu - etiketler_bir_sicak_vektor)
    
    # Güvenilirlik katsayılarının sayısal türde olduğundan emin olun
    if guvenilirlik is not None:
        guvenilirlik = np.asarray(guvenilirlik, dtype=np.float64)  # guvenilirlik float64 türüne dönüştürülüyor

    # Delta2 ile guvenilirlik katsayılarını çarpma işlemi
    if guvenilirlik is not None:
        delta2 = delta2 * guvenilirlik[:, np.newaxis]  # Delta üzerinde güvenilirlik katsayılarıyla ölçeklendirme
    
    # Güncellenmiş delta2'yi kullanarak türevleri hesaplama
    gizli_katman_agirlik_turevi = np.matmul(gizli_katman_aktivasyonu.T, delta2)
    delta1 = np.matmul(delta2, agirliklar[1].T) * (gizli_katman_giris > 0)
    giris_katman_agirlik_turevi = np.matmul(girdi_verisi.T, delta1)
    
    # Düzenleme katsayısı ile ağırlık türevlerini güncelleme
    agirlik_turevleri = [giris_katman_agirlik_turevi / numune_sayisi + duzenleme_katsayisi * agirliklar[0], 
                         gizli_katman_agirlik_turevi / numune_sayisi + duzenleme_katsayisi * agirliklar[1]]
    bias_turevleri = [np.mean(delta1, axis=0), np.mean(delta2, axis=0)]
    
    return agirlik_turevleri, bias_turevleri

In [25]:
y_veri = df['doctor_speciality'].values
type(y_veri), y_veri.shape, y_veri[:5]

(numpy.ndarray, (124748,), array([0, 0, 0, 1, 1], dtype=object))

In [26]:
g_veri = df['doctor_title'].values
type(g_veri), g_veri.shape, g_veri[:5]

(numpy.ndarray, (124748,), array([7, 6, 6, 6, 6], dtype=object))

In [22]:
X_veri_list = []
X_veri_list.append(embed_text("Merhabalar, benim adım Ali. Bugün hava çok güzel."))

X_veri = np.array(X_veri_list)
X_veri.shape

(1, 768)

In [23]:
# map text to embeddings and convert to numpy
X_veri_list = []
for text in df['text']:
    X_veri_list.append(embed_text(text))

X_veri = np.array(X_veri_list)
type(X_veri), X_veri.shape, X_veri[0].shape

(numpy.ndarray, (124748, 768), (768,))

In [29]:
X_test = X_veri[:1000]
y_test = y_veri[:1000]
g_test = g_veri[:1000]

X_egitim = X_veri[1000:]
y_egitim = y_veri[1000:]
g_egitim = g_veri[1000:]

print(X_egitim.shape, y_egitim.shape)
print(X_test.shape, y_test.shape)
print(g_test.shape, g_egitim.shape)

(123748, 768) (123748,)
(1000, 768) (1000,)
(1000,) (123748,)


In [30]:
X_egitim[0].shape

(768,)

In [19]:
# {“uzman-doktor”: 70991, “profesor”: 38091, “docent”: 12897, “dr-ogr-uyesi”: 2420, “doktor”: 349} 928717
# 6, 10, 8, 7, 5


guvenilirlik = {
    6: 70991 * 6 / 185743.4,
    10: 38091 * 10 / 185743.4,
    8: 12897 * 8 / 185743.4,
    7: 2420 * 7 / 185743.4,
    5: 349 * 5 / 185743.4
}
guvenilirlik

{6: 2.293195882060951,
 10: 2.05073235442013,
 8: 0.5554759953785706,
 7: 0.09120108709111603,
 5: 0.009394681049232436}

In [72]:
# Parametrelerin tanımlanması
ogrenme_orani_eta = 5e-1  # Öğrenme oranı
duzenleme_katsayisi_alpha = 1e-6  # Düzenleme katsayısı
rmsprop_gamma = 0.99  # RMSprop için gamma değeri
rmsprop_eps = 1e-3  # RMSprop için epsilon değeri
toplam_iterasyon_sayisi = 300  # Gradyan iniş algoritmasının iterasyon sayısı
gizli_katman_noron_sayisi = 256  # Gizli katmandaki nöron sayısı
girdi_ozellik_sayisi = X_egitim.shape[1]  # Bir resimdeki piksel sayısı
sinif_sayisi = 2  # Sınıf sayısı

In [43]:
list(X_test[15])

[0.0038652422,
 0.012153784,
 0.0016007624,
 0.012870334,
 0.012013456,
 0.026700977,
 -0.01077178,
 -0.029163785,
 0.014100477,
 -0.0076350453,
 0.006495018,
 0.0069767446,
 0.014066488,
 -0.003869129,
 0.014449706,
 0.0021180848,
 0.008775139,
 0.00742837,
 0.0021718235,
 -0.009240747,
 -0.025006,
 0.0017618749,
 0.0036908486,
 8.8611916e-05,
 -0.015158061,
 -0.00618413,
 -0.010778325,
 0.002779657,
 -0.008532285,
 0.028836789,
 0.011027516,
 -0.0009777647,
 0.006853496,
 -0.0052956347,
 -0.00051327574,
 0.0013211175,
 0.015337447,
 0.004009064,
 -0.0010593038,
 0.010944274,
 -0.029160231,
 0.034601517,
 0.00547982,
 0.0031273533,
 0.006833258,
 -0.030714542,
 -0.0027795534,
 -0.013043098,
 0.008413021,
 0.00833334,
 0.0002738147,
 0.0007301546,
 -0.012476387,
 -0.03616415,
 -0.02337482,
 -0.021534009,
 -0.007463403,
 -0.009415559,
 -0.024384322,
 -0.01910168,
 0.010278439,
 0.015908571,
 0.0013542258,
 -0.0114051225,
 -0.0009352055,
 0.01403055,
 -0.0008700271,
 -0.019790815,
 0.008

In [82]:
# Başlangıç değerleri
np.random.seed(42)
# Ağırlık ve biasların başlangıç değerleri
agirliklar = [np.random.randn(girdi_ozellik_sayisi, gizli_katman_noron_sayisi), 
              np.random.randn(gizli_katman_noron_sayisi, sinif_sayisi)]
biaslar = [np.random.randn(gizli_katman_noron_sayisi)]

In [78]:
agirliklar[0].shape, agirliklar[1].shape, biaslar[0].shape

((768, 256), (256, 2), (256,))

In [80]:
# Güvenilirlik katsayılarını belirlemek için tanımlama
guvenilirlik = {
    6: 70991 * 6 / 185743.4,
    10: 38091 * 10 / 185743.4,
    8: 12897 * 8 / 185743.4,
    7: 2420 * 7 / 185743.4,
    5: 349 * 5 / 185743.4
}

# g_egitim'in her bir elemanını guvenilirlik katsayıları ile ölçeklendirmek için katsayıları uygulama
g_egitim_olceklendirilmis = np.array([guv / 10 for guv in g_egitim])
g_egitim_olceklendirilmis[:5], g_egitim[:5]

(array([1. , 0.6, 1. , 0.6, 0.6]), array([10, 6, 10, 6, 6], dtype=object))

In [83]:
%%time

# Başlangıç değerleri
gizli_katman_agirliklari_gW0 = cikis_katman_agirliklari_gW1 = gizli_katman_biaslari_gb0 = 1

# Eğitim döngüsü
for iterasyon in range(toplam_iterasyon_sayisi):
    # Ağırlık ve bias türevlerini geri yayılım fonksiyonu ile hesapla
    agirlik_turevleri, bias_turevleri = geri_yayilim(
        agirliklar, biaslar, X_egitim, y_egitim, duzenleme_katsayisi_alpha, guvenilirlik=g_egitim_olceklendirilmis
    )
    
    # RMSprop güncellemeleri ile ağırlıkların ayarlanması
    gizli_katman_agirliklari_gW0 = rmsprop_gamma * gizli_katman_agirliklari_gW0 + (1 - rmsprop_gamma) * np.sum(agirlik_turevleri[0]**2)
    ogrenme_orani_etaW0 = ogrenme_orani_eta / np.sqrt(gizli_katman_agirliklari_gW0 + rmsprop_eps)
    agirliklar[0] -= ogrenme_orani_etaW0 * agirlik_turevleri[0]

    cikis_katman_agirliklari_gW1 = rmsprop_gamma * cikis_katman_agirliklari_gW1 + (1 - rmsprop_gamma) * np.sum(agirlik_turevleri[1]**2)
    ogrenme_orani_etaW1 = ogrenme_orani_eta / np.sqrt(cikis_katman_agirliklari_gW1 + rmsprop_eps)
    agirliklar[1] -= ogrenme_orani_etaW1 * agirlik_turevleri[1]

    # Bias güncellemeleri
    gizli_katman_biaslari_gb0 = rmsprop_gamma * gizli_katman_biaslari_gb0 + (1 - rmsprop_gamma) * np.sum(bias_turevleri[0]**2)
    ogrenme_orani_etab0 = ogrenme_orani_eta / np.sqrt(gizli_katman_biaslari_gb0 + rmsprop_eps)
    biaslar[0] -= ogrenme_orani_etab0 * bias_turevleri[0]
    
    if iterasyon % 5 == 0:
      # Durum kontrolü 1: Kayıp ve doğruluk hesaplama
      tahminler = tahmin_et(X_egitim, agirliklar, biaslar)
      print("Cross-entropy kaybı", iterasyon + 1, "iterasyondan sonra {:.8}".format(
            kayip_hesapla(tahminler, y_egitim)))
      print("Eğitim doğruluğu", iterasyon + 1, "iterasyondan sonra {:.4%}".format( 
            np.mean(np.argmax(tahminler, axis=1) == y_egitim)))
      
      # Durum kontrolü 2: RMSprop değerlerinin çıktısını yazdırma
      print("gizli_katman_agirliklari_gW0={:.4f} cikis_katman_agirliklari_gW1={:.4f} gizli_katman_biaslari_gb0={:.4f}\n"
            "ogrenme_orani_etaW0={:.4f} ogrenme_orani_etaW1={:.4f} ogrenme_orani_etab0={:.4f}"
            .format(gizli_katman_agirliklari_gW0, cikis_katman_agirliklari_gW1, gizli_katman_biaslari_gb0, 
                    ogrenme_orani_etaW0, ogrenme_orani_etaW1, ogrenme_orani_etab0))
      
      # Durum kontrolü 3: Ağırlık ve bias türevlerinin normlarını yazdırma
      print("|agirlik_turevleri[0]|={:.5f} |agirlik_turevleri[1]|={:.5f} |bias_turevleri[0]|={:.5f}"
            .format(np.linalg.norm(agirlik_turevleri[0]), np.linalg.norm(agirlik_turevleri[1]), np.linalg.norm(bias_turevleri[0])), "\n")
      
      # RMSprop değerlerini sıfırlama
      gizli_katman_agirliklari_gW0 = cikis_katman_agirliklari_gW1 = gizli_katman_biaslari_gb0 = 1

# Son tahminler ve sonuçların yazdırılması
son_tahminler = tahmin_et(X_egitim, agirliklar, biaslar)
print("Son cross-entropy kaybı {:.8}".format(kayip_hesapla(son_tahminler, y_egitim)))
print("Son eğitim doğruluğu {:.4%}".format(np.mean(np.argmax(son_tahminler, axis=1) == y_egitim)))


Cross-entropy kaybı 1 iterasyondan sonra 27.266498
Eğitim doğruluğu 1 iterasyondan sonra 53.9597%
gizli_katman_agirliklari_gW0=1.0170 cikis_katman_agirliklari_gW1=1.2003 gizli_katman_biaslari_gb0=1.2122
ogrenme_orani_etaW0=0.4956 ogrenme_orani_etaW1=0.4562 ogrenme_orani_etab0=0.4539
|agirlik_turevleri[0]|=1.64190 |agirlik_turevleri[1]|=4.58626 |bias_turevleri[0]|=4.71374 

Cross-entropy kaybı 6 iterasyondan sonra 18.27686
Eğitim doğruluğu 6 iterasyondan sonra 53.9597%
gizli_katman_agirliklari_gW0=1.0526 cikis_katman_agirliklari_gW1=1.9047 gizli_katman_biaslari_gb0=1.8102
ogrenme_orani_etaW0=0.4871 ogrenme_orani_etaW1=0.3622 ogrenme_orani_etab0=0.3715
|agirlik_turevleri[0]|=1.55394 |agirlik_turevleri[1]|=4.75734 |bias_turevleri[0]|=4.51635 

Cross-entropy kaybı 11 iterasyondan sonra 1.7495298
Eğitim doğruluğu 11 iterasyondan sonra 46.4517%
gizli_katman_agirliklari_gW0=1.0430 cikis_katman_agirliklari_gW1=1.8725 gizli_katman_biaslari_gb0=1.7321
ogrenme_orani_etaW0=0.4894 ogrenme_orani_eta

In [84]:
# predictions
y_pred_test = np.argmax(tahmin_et(X_test, agirliklar, biaslar), axis=1)
print("Test accuracy is {:.4%}".format(np.mean(y_pred_test == y_test)))

Test accuracy is 64.8000%
